In [0]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_unixtime, to_date, col, lower, regexp_replace, dense_rank, when, expr, desc
from pyspark.conf import SparkConf
from pyspark.sql.window import Window
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
# Cargamos los datos que guardamos del proceso etl de entrenamiento
train_df = spark.read.format("parquet").load("/mnt/backupaws/Datos_ML_BAR/Datos_Bar_entrenamiento/part-00000-tid-7141405282916243732-b17bb745-8483-4133-93dc-d23f740f7603-2214-1-c000.snappy.parquet")

In [0]:
# Cargamos los datos que guardamos del proceso etl de entrenamiento
test_df = spark.read.format("parquet").load("/mnt/backupaws/Datos_ML_BAR/Datos_Bar_testeo/part-00000-tid-6453973100730320895-f76a13df-0423-4b94-a042-3de55daa595d-2231-1-c000.snappy.parquet")

In [0]:
als = ALS(maxIter=100, regParam=0.1, userCol="id_name", itemCol="id_name_empresa", ratingCol="rating",
          coldStartStrategy="drop", blockSize=4096)
als = als.setPredictionCol("prediction")

In [0]:
model = als.fit(train_df)

In [0]:
# Definir evaluador
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [0]:
predictions = model.transform(test_df)

In [0]:
# maxiter =50 --- 2.532
# maxiter = 100 --- 2.0996
predictions = model.transform(test_df)
rmse = evaluator.evaluate(predictions)
print("RMSE en conjunto de pruebas: {:.4f}".format(rmse))

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-239659893837749>:4
      1 # maxiter =50 --- 2.532
      2 # maxiter = 100 --- 2.0996
      3 predictions = model.transform(test_df)
----> 4 rmse = evaluator.evaluate(predictions)
      5 print("RMSE en conjunto de pruebas: {:.4f}".format(rmse))

File /databricks/spark/python/pyspark/ml/evaluation.py:111, in Evaluator.evaluate(self, dataset, params)
    109         return self.copy(params)._evaluate(dataset)
    110     else:
--> 111         return self._evaluate(dataset)
    112 else:
    113     raise TypeError("Params must be a param map but got %s." % type(params))

File /databricks/spark/python/pyspark/ml/evaluation.py:148, in JavaEvaluator._evaluate(self, dataset)
    146 self._transfer_params_to_java()
    147 assert self._java_obj is not None
--> 148 return self._java_obj.evaluate(dataset._jdf)

File /databricks/sp

In [0]:
# Predicciones para cada usuario
userRecs = model.recommendForAllUsers(100)

# Selección de las 10 películas más recomendadas para cada usuario
userRecs = userRecs.select(
    col("id_name"),
    expr("explode(recommendations)").alias("recommendation")
).select(
    col("id_name"),
    col("recommendation.id_name_empresa"),
    col("recommendation.rating")
)

# Ordenamiento de las películas según su predicción
userRecs = userRecs.orderBy(
    col("id_name"), desc("rating")
)

In [0]:
userRecs = userRecs.coalesce(1)

In [0]:
userRecs.write.mode("overwrite").parquet("/mnt/backupaws/metadata_coffe/Users_recomendations")